In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [68]:
import xml.etree.ElementTree as ET  
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from statistics import mean
import math

In [3]:
input_path='/content/drive/MyDrive/Datasets/query/1/1.inkml'

In [4]:
def print_description(root):
  for x in root[0]:
    print(x.text)

In [5]:
def extract_xy(input_path):
  
  tree = ET.parse(input_path)
  root = tree.getroot()
  
  print_description(root)

  a=[]
  b=[]
  all_x=[]
  all_y=[]
  x_coordinates = []
  y_coordinates = []
  for i in range(1,len(root)):
    for x in root[i]:
      if (x.text=='\n\t\t\t'):
        continue
      else:
        l = list(map(str,x.text.split(',')))
        l = [x.strip() for x in l]
        for x in l:
          xy = list(map(int,x.split(' ')))
          x_coordinates.append(xy[0])
          y_coordinates.append(xy[1])
        a.append(x_coordinates)
        b.append(y_coordinates)
        x_coordinates = []
        y_coordinates = []  
  for i in range(len(a)):
    for j in range(len(a[i])):
      all_x.append(a[i][j])
      all_y.append(b[i][j])
  return(a,b,all_x,all_y)       

In [6]:
[tracewise_x,tracewise_y,whole_x,whole_y]=extract_xy(input_path)

cursive online handwriting 
Text
English
AAR
113
25
186
 FAC100z Offer Committee employees living hopes community benefits currently month assistant report extended auditor funds months housing new extended away long top economy attorney funds


In [7]:
time_stamp = np.arange(0,len(whole_x)*0.02,0.02)

In [8]:
def relative_x_y(whole_x,whole_y):
  relative_x=[0]
  relative_y=[0]
  for i in range(1,len(whole_x)):
    relative_x.append(whole_x[i]-mean(whole_x[:i]))
    relative_y.append(whole_y[i]-mean(whole_y[:i]))
  return(relative_x,relative_y)

In [9]:
[relative_x,relative_y]=relative_x_y(whole_x,whole_y)

In [10]:
def speed(whole_x,whole_y):
  speed=[0]
  for i in range(1,len(whole_x)):
    speed.append(((whole_x[i]-whole_x[i-1])**2-(whole_y[i]-whole_y[i-1])**2)**(0.5)/0.02)
  return(speed)

In [11]:
speed=speed(whole_x,whole_y)

In [12]:
def speed_x_y(tracewise_x,tracewise_y):
  speed_x=[]
  speed_y=[]
  tracewise_speed_x=[]
  tracewise_speed_y=[]
  temp_x=[0]
  temp_y=[0]
  for i in range(len(tracewise_x)):
    for j in range(1,len(tracewise_x[i])):
      temp_x.append((tracewise_x[i][j]-tracewise_x[i][j-1])/0.02)
      temp_y.append((tracewise_y[i][j]-tracewise_y[i][j-1])/0.02)
    tracewise_speed_x.append(temp_x)
    tracewise_speed_y.append(temp_y)
    temp_x=[0]
    temp_y=[0]
  for i in range(len(tracewise_speed_x)):
    for j in range(len(tracewise_speed_x[i])):
      speed_x.append(tracewise_speed_x[i][j])
      speed_y.append(tracewise_speed_y[i][j])
  return(speed_x,speed_y,tracewise_speed_x,tracewise_speed_y)

In [13]:
[speed_x,speed_y,tracewise_speed_x,tracewise_speed_y]=speed_x_y(tracewise_x,tracewise_y)

In [14]:
def acceleration(speed):
  acc=[0]
  for i in range(1,len(speed)):
    acc.append((speed[i]-speed[i-1])/0.02)
  return(acc)

In [15]:
acceleration = acceleration(speed)

In [16]:
def acceleration_x_y(tracewise_speed_x,tracewise_speed_y):
  acc_x=[0]
  acc_y=[0]
  tracewise_acceleration_x=[]
  tracewise_acceleration_y=[]
  acceleration_x=[]
  acceleration_y=[]
  for i in range(len(tracewise_speed_x)):
    for j in range(1,len(tracewise_speed_x[i])):
      acc_x.append((tracewise_speed_x[i][j]-tracewise_speed_x[i][j-1])/0.02)
      acc_y.append((tracewise_speed_y[i][j]-tracewise_speed_y[i][j-1])/0.02)
    tracewise_acceleration_x.append(acc_x)
    tracewise_acceleration_y.append(acc_y)
    acc_x=[0]
    acc_y=[0]
  for i in range(len(tracewise_acceleration_x)):
    for j in range(len(tracewise_acceleration_x[i])):
      acceleration_x.append(tracewise_acceleration_x[i][j])
      acceleration_y.append(tracewise_acceleration_y[i][j])
  
  return(tracewise_acceleration_x,tracewise_acceleration_y,acceleration_x,acceleration_y)

In [17]:
[tracewise_acceleration_x,tracewise_acceleration_y,acceleration_x,acceleration_y]=acceleration_x_y(tracewise_speed_x,tracewise_speed_y)

In [18]:
def log_curvature_radius(vx,vy,ax,ay):
  rad=[]
  for i in range(len(vx)):
    nr=(vx[i]*ay[i]-ax[i]-vy[i])
    dr=(vx[i]**2+vy[i]**2)**(1.5)
    if (nr==0 and dr==0):
      rad.append(0)
    else:
      rad.append(nr/dr)
  return(rad)

In [19]:
log_curvature_radius = log_curvature_radius(speed_x,speed_y,acceleration_x,acceleration_y)

In [20]:
def writing_direction_x_y(whole_x,whole_y):
  cos=[0]
  sin=[0]
  for i in range(1,len(whole_x)):
    l=((whole_x[i]-whole_x[i-1])**2+(whole_y[i]-whole_y[i-1])**2)**(0.5)
    cos.append((whole_x[i]-whole_x[i-1])/l)
    sin.append((whole_y[i]-whole_y[i-1])/l)
  return(cos,sin)

In [21]:
[writing_direction_x,writing_direction_y]=writing_direction_x_y(whole_x,whole_y)

In [22]:
def curvature_x_y(wx,wy):
  cur_x=[0]
  cur_y=[0]
  for i in range(1,len(wx)):
    cur_x.append(np.cos(wx[i-1])*np.cos(wx[i])+np.sin(wx[i-1])*np.sin(wx[i]))
    cur_y.append(np.cos(wx[i-1])*np.sin(wx[i])-np.sin(wx[i-1])*np.cos(wx[i]))
  return(cur_x,cur_y)

In [23]:
[curvature_x,curvature_y]=curvature_x_y(writing_direction_x,writing_direction_y)

In [50]:
def vicinity_aspect(tracewise_x,tracewise_y):
  temp_x=[]
  temp_y=[]
  left_x=[]
  right_x=[]
  left_y=[]
  right_y=[]
  var=[]
  for i in range(len(tracewise_x)):
    for j in range(len(tracewise_x[i])):
      if len(tracewise_x[i])>=11:
        if len(tracewise_x[i][:j])>5:
          left_x=tracewise_x[i][j-5:j]
        elif len(tracewise_x[i][:j])<=5:
          left_x=tracewise_x[i][:j]
        if len(tracewise_x[i][j+1:])>5:
          right_x=tracewise_x[i][j+1:j+6]
        elif len(tracewise_x[i][j+1:])<=5:
          right_x=tracewise_x[i][j+1:]
      elif len(tracewise_x[i])<11:
        if len(tracewise_x[i][:j])>5:
          left_x=tracewise_x[i][j-5:j]
        elif len(tracewise_x[i][:j])<=5:
          left_x=tracewise_x[i][:j]
        if len(tracewise_x[i][j+1:])>5:
          right_x=tracewise_x[i][j+1:j+6]
        elif len(tracewise_x[i][j+1:])<=5:
          right_x=tracewise_x[i][j+1:]

      if len(tracewise_y[i])>=11:
        if len(tracewise_y[i][:j])>5:
          left_y=tracewise_y[i][j-5:j]
        elif len(tracewise_y[i][:j])<=5:
          left_y=tracewise_y[i][:j]
        if len(tracewise_y[i][j+1:])>5:
          right_y=tracewise_y[i][j+1:j+6]
        elif len(tracewise_y[i][j+1:])<=5:
          right_y=tracewise_y[i][j+1:]
      elif len(tracewise_y[i])<11:
        if len(tracewise_y[i][:j])>5:
          left_y=tracewise_y[i][j-5:j]
        elif len(tracewise_y[i][:j])<=5:
          left_y=tracewise_y[i][:j]
        if len(tracewise_y[i][j+1:])>5:
          right_y=tracewise_y[i][j+1:j+6]
        elif len(tracewise_y[i][j+1:])<=5:
          right_y=tracewise_y[i][j+1:]

      left_x.append(tracewise_x[i][j])
      left_y.append(tracewise_y[i][j])
      temp_x=left_x+right_x
      temp_y=left_y+right_y
      dx=max(temp_x)-min(temp_x)
      dy=max(temp_y)-min(temp_y)
      if dx+dy==0:
        var.append(0)
      else:
        var.append((dy-dx)/(dx+dy))
      #print(temp_x)
      #print(temp_y)
  return(var)

In [51]:
vicinity_aspect_ratio=vicinity_aspect(tracewise_x,tracewise_y)

In [61]:
def vicinity_curliness(tracewise_x,tracewise_y):
  temp_x=[]
  temp_y=[]
  left_x=[]
  right_x=[]
  left_y=[]
  right_y=[]
  vc=[]
  for i in range(len(tracewise_x)):
    for j in range(len(tracewise_x[i])):
      if len(tracewise_x[i])>=11:
        if len(tracewise_x[i][:j])>5:
          left_x=tracewise_x[i][j-5:j]
        elif len(tracewise_x[i][:j])<=5:
          left_x=tracewise_x[i][:j]
        if len(tracewise_x[i][j+1:])>5:
          right_x=tracewise_x[i][j+1:j+6]
        elif len(tracewise_x[i][j+1:])<=5:
          right_x=tracewise_x[i][j+1:]
      elif len(tracewise_x[i])<11:
        if len(tracewise_x[i][:j])>5:
          left_x=tracewise_x[i][j-5:j]
        elif len(tracewise_x[i][:j])<=5:
          left_x=tracewise_x[i][:j]
        if len(tracewise_x[i][j+1:])>5:
          right_x=tracewise_x[i][j+1:j+6]
        elif len(tracewise_x[i][j+1:])<=5:
          right_x=tracewise_x[i][j+1:]

      if len(tracewise_y[i])>=11:
        if len(tracewise_y[i][:j])>5:
          left_y=tracewise_y[i][j-5:j]
        elif len(tracewise_y[i][:j])<=5:
          left_y=tracewise_y[i][:j]
        if len(tracewise_y[i][j+1:])>5:
          right_y=tracewise_y[i][j+1:j+6]
        elif len(tracewise_y[i][j+1:])<=5:
          right_y=tracewise_y[i][j+1:]
      elif len(tracewise_y[i])<11:
        if len(tracewise_y[i][:j])>5:
          left_y=tracewise_y[i][j-5:j]
        elif len(tracewise_y[i][:j])<=5:
          left_y=tracewise_y[i][:j]
        if len(tracewise_y[i][j+1:])>5:
          right_y=tracewise_y[i][j+1:j+6]
        elif len(tracewise_y[i][j+1:])<=5:
          right_y=tracewise_y[i][j+1:]

      left_x.append(tracewise_x[i][j])
      left_y.append(tracewise_y[i][j])
      temp_x=left_x+right_x
      temp_y=left_y+right_y
      dx=max(temp_x)-min(temp_x)
      dy=max(temp_y)-min(temp_y)
      
      dist=[]
      for k in range(1,len(temp_x)):
        dist.append(((temp_x[k]-temp_x[k-1])**2+(temp_y[k]-temp_y[k-1])**2)**(0.5))
      if dx+dy==0:
        vc.append(0)
      else:
        vc.append(sum(dist)/max(dx,dy))
      dist=[]
  return(vc)

In [65]:
vicinity_curliness = vicinity_curliness(tracewise_x,tracewise_y)

In [69]:
def vicinity_slope(tracewise_x,tracewise_y):
  temp_x=[]
  temp_y=[]
  left_x=[]
  right_x=[]
  left_y=[]
  right_y=[]
  vs_cos=[]
  vs_sin=[]
  for i in range(len(tracewise_x)):
    for j in range(len(tracewise_x[i])):
      if len(tracewise_x[i])>=11:
        if len(tracewise_x[i][:j])>5:
          left_x=tracewise_x[i][j-5:j]
        elif len(tracewise_x[i][:j])<=5:
          left_x=tracewise_x[i][:j]
        if len(tracewise_x[i][j+1:])>5:
          right_x=tracewise_x[i][j+1:j+6]
        elif len(tracewise_x[i][j+1:])<=5:
          right_x=tracewise_x[i][j+1:]
      elif len(tracewise_x[i])<11:
        if len(tracewise_x[i][:j])>5:
          left_x=tracewise_x[i][j-5:j]
        elif len(tracewise_x[i][:j])<=5:
          left_x=tracewise_x[i][:j]
        if len(tracewise_x[i][j+1:])>5:
          right_x=tracewise_x[i][j+1:j+6]
        elif len(tracewise_x[i][j+1:])<=5:
          right_x=tracewise_x[i][j+1:]

      if len(tracewise_y[i])>=11:
        if len(tracewise_y[i][:j])>5:
          left_y=tracewise_y[i][j-5:j]
        elif len(tracewise_y[i][:j])<=5:
          left_y=tracewise_y[i][:j]
        if len(tracewise_y[i][j+1:])>5:
          right_y=tracewise_y[i][j+1:j+6]
        elif len(tracewise_y[i][j+1:])<=5:
          right_y=tracewise_y[i][j+1:]
      elif len(tracewise_y[i])<11:
        if len(tracewise_y[i][:j])>5:
          left_y=tracewise_y[i][j-5:j]
        elif len(tracewise_y[i][:j])<=5:
          left_y=tracewise_y[i][:j]
        if len(tracewise_y[i][j+1:])>5:
          right_y=tracewise_y[i][j+1:j+6]
        elif len(tracewise_y[i][j+1:])<=5:
          right_y=tracewise_y[i][j+1:]

      left_x.append(tracewise_x[i][j])
      left_y.append(tracewise_y[i][j])
      temp_x=left_x+right_x
      temp_y=left_y+right_y

      if (temp_x[-1]-temp_x[0])==0:
        vs_cos.append(1)
        vs_sin.append(0)
      else:
        slope = (temp_y[-1]-temp_y[0])/(temp_x[-1]-temp_x[0])
        theta = math.atan(slope)
        vs_cos.append(math.sin(theta))
        vs_sin.append(math.cos(theta))
      
  return(vs_cos,vs_sin)

In [70]:
[vs_cos,vs_sin]=vicinity_slope(tracewise_x,tracewise_y)

In [71]:
print(len(vs_cos),len(vs_sin))

8839 8839


In [72]:
print(vs_cos)
print(vs_sin)

[-0.9486832980505138, -0.9647638212377322, -0.9743911956946198, -0.9486832980505138, -0.9544799780350297, -0.9635179096299407, -0.932004671541296, -0.932004671541296, -0.9593655015712705, -0.9635179096299407, -0.9761870601839527, -0.9761870601839527, -0.9615239476408232, -0.9805806756909201, -0.9615239476408232, -0.7071067811865475, -0.9284766908852593, -0.9883716976506172, -0.978549784986749, -0.9899494936611666, -0.9918816646858506, -0.9950371902099892, -0.9961747239492219, -0.9940716917543757, -0.9948934062566149, -0.9956726957313967, -0.9980031131477184, -0.9980525784828885, -0.9993314265864826, -0.9992976157091806, -0.9996628455736942, -0.9985021523140876, -0.9982743731749959, -0.9994449069791543, -0.9993394979464322, -0.9991330730923519, -0.9985422732775083, -0.997458699830735, -0.9938837346736189, 1, 1, -0.9938837346736189, -0.9917778666340251, -0.992645877550526, -0.9954090291002811, -0.993686724241462, -0.9947432988901834, -0.9924856303405769, -0.993238371471016, -0.9930611231